In [1]:
import pandas as pd
import numpy as np

loans = pd.read_csv("../course_3_data/lending-club-data.csv")

/home/prachi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans',axis=1)

In [3]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]

target = 'safe_loans'
loans = loans[features+[target]]

In [4]:
train_idx=pd.read_json("../course_3_data/module-6-assignment-train-idx.json")
validation_idx=pd.read_json("../course_3_data/module-6-assignment-validation-idx.json")

In [5]:
train_data = loans.iloc[train_idx[0]]
validation_data = loans.iloc[validation_idx[0]]

In [6]:
train_data=pd.get_dummies(train_data)
validation_data=pd.get_dummies(validation_data)

In [7]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(frac=percentage)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46300


In [8]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    if len(data)<=min_node_size:
        return True
    else:
        return False

In [9]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    ## YOUR CODE HERE
    return error_before_split-error_after_split

In [10]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE  
    count_safe=len([i for i in labels_in_node if i==1])
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE 
    count_risky=len([i for i in labels_in_node if i==-1])
    
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE  
    if count_safe>count_risky:
        return count_risky
    else:
        return count_safe

In [11]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split = data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])          

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes+right_mistakes)/num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_feature=feature
            best_error=error
        
    return best_feature # Return the best feature we found
        

In [12]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True   }   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =1       ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

In [13]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print("Stopping condition 1 reached. All data points have the same target value.")              
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features== []:
        print("Stopping condition 2 reached. No remaining features." )               
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data,min_node_size):      ## YOUR CODE HERE 
        print ("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)   ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes =    intermediate_node_num_mistakes(left_split[target])
    right_mistakes =   intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    print(error_before_split,error_after_split)
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split,error_after_split)<=min_error_reduction:        ## YOUR CODE HERE
        print("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values)  ## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [14]:
features=list(train_data.columns[1:])
features


['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

In [15]:
train_data

safe_loans  grade_A  grade_B  grade_C  grade_D  grade_E  grade_F  \
1               -1        0        0        1        0        0        0   
6               -1        0        0        0        0        0        1   
7               -1        0        1        0        0        0        0   
10              -1        0        0        1        0        0        0   
12              -1        0        1        0        0        0        0   
...            ...      ...      ...      ...      ...      ...      ...   
122572           1        0        1        0        0        0        0   
122575           1        0        0        1        0        0        0   
122588           1        1        0        0        0        0        0   
122599           1        0        0        0        0        0        0   
122603           1        0        0        0        1        0        0   

        grade_G  term_ 36 months  term_ 60 months  ...  emp_length_10+ years  \
1             0                0                1  ...                     0   
6             0                0                1  ...                     0   
7             0                0                1  ...                     0   
10            0                1                0  ...                     0   
12            0                1                0  ...                     0   
...         ...              ...              ...  ...                   ...   
122572        0                1                0  ...                     0   
122575        0                1                0  ...                     0   
122588        0                1                0  ...                     0   
122599        1                0                1  ...                     1   
122603        0                1                0  ...                     1   

        emp_length_2 years  emp_length_3 years  emp_length_4 years  \
1                        0                   0                   0   
6                        0                   0                   1   
7                        0                   0                   0   
10                       0                   0                   0   
12                       0                   1                   0   
...                    ...                 ...                 ...   
122572                   0                   0                   0   
122575                   0                   0                   0   
122588                   0                   1                   0   
122599                   0                   0                   0   
122603                   0                   0                   0   

        emp_length_5 years  emp_length_6 years  emp_length_7 years  \
1                        0                   0                   0   
6                        0                   0                   0   
7                        0                   0                   0   
10                       0                   0                   0   
12                       0                   0                   0   
...                    ...                 ...                 ...   
122572                   0                   0                   0   
122575                   1                   0                   0   
122588                   0                   0                   0   
122599                   0                   0                   0   
122603                   0                   0                   0   

        emp_length_8 years  emp_length_9 years  emp_length_< 1 year  
1                        0                   0                    1  
6                        0                   0                    0  
7                        0                   0                    1  
10                       0                   0                    1  
12                       0                   0                    0  
...                    ...      

In [16]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
0.4963464431549538 0.4216365785514722
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
0.34923560663558495 0.34674184104955
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
0.34630563472922604 0.34630563472922604
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
0.38613861386138615 0.37623762376237624
Split on feature emp_length_< 1 year. (90, 11)
--------------------------------------------------------------------
Subtree, depth = 3 (90 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------

In [17]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
0.4963464431549538 0.4216365785514722
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
0.34923560663558495 0.34674184104955
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
0.34630563472922604 0.34630563472922604
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
0.33415902898191724 0.33415902898191724
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
0.30319510537049627 0.30319510537049627
Split on feature grade_D. (3826, 2058)
-------------------------------------------------------------

0.4056483835815474 0.3992008717762441
Split on feature grade_F. (21666, 358)
--------------------------------------------------------------------
Subtree, depth = 4 (21666 data points).
0.4008123326871596 0.39831994830610173
Split on feature grade_C. (14444, 7222)
--------------------------------------------------------------------
Subtree, depth = 5 (14444 data points).
0.349349210744946 0.34623373026862364
Split on feature grade_G. (14347, 97)
--------------------------------------------------------------------
Subtree, depth = 6 (14347 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (97 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (7222 data points).
0.4962614234284132 0.4745222929936306
Split on feature home_ownership_MORTGAGE. (4303, 2919)
----------------

In [18]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)
               ### YOUR CODE HERE

In [19]:
print(validation_data.iloc[0])
print('Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0]))


safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 24, dtype: int64
Predicted class: -1 


In [20]:
classify(my_decision_tree_new, validation_data.iloc[0], annotate = True)

Split on term_ 36 months = 0
Split on grade_A = 0
At leaf, predicting -1


-1

In [21]:
classify(my_decision_tree_old, validation_data.iloc[0], annotate = True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
Split on grade_E = 0
At leaf, predicting -1


-1

In [22]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
#     prediction = data.apply(lambda x: classify(tree, x))
    prediction=[]
    for i in range(len(data)):
        prediction.append(classify(tree,data.iloc[i]))
    
    # Once you've made the predictions, calculate the classification error and return it
    false=sum(data['safe_loans']!=prediction)
    mistakes=false/len(data)
    
    return mistakes
    ## YOUR CODE HERE

In [23]:
evaluate_classification_error(my_decision_tree_new, validation_data)

0.37774666092201636

In [24]:
evaluate_classification_error(my_decision_tree_old, validation_data)

0.37774666092201636

In [25]:
#too small
model_1=decision_tree_create(train_data, features, 'safe_loans', max_depth = 2,min_node_size = 0, min_error_reduction=-1)

#just right
model_2=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)


#may be too large
model_3=decision_tree_create(train_data, features, 'safe_loans', max_depth = 14,min_node_size = 0, min_error_reduction=-1)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
0.4963464431549538 0.4216365785514722
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
0.34923560663558495 0.34674184104955
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
0.4454840898539338 0.4175922288489697
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data po

0.4454840898539338 0.4175922288489697
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
0.417725321888412 0.4039055793991416
Split on feature grade_E. (22024, 1276)
--------------------------------------------------------------------
Subtree, depth = 3 (22024 data points).
0.4056483835815474 0.3992008717762441
Split on feature grade_F. (21666, 358)
--------------------------------------------------------------------
Subtree, depth = 4 (21666 data points).
0.4008123326871596 0.39831994830610173
Split on feature grade_C. (14444, 7222)
--------------------------------------------------------------------
Subtree, depth = 5 (14444 data points).
0.349349210744946 0.34623373026862364
Split on feature grade_G. (14347, 97)
--------------------------------------------------------------------
Subtree, depth = 6 (14347 data points).
Early stopping condition 1 reached. Reached maximum depth.
---------

0.2057142857142857 0.2057142857142857
Split on feature home_ownership_OWN. (142, 33)
--------------------------------------------------------------------
Subtree, depth = 12 (142 data points).
0.20422535211267606 0.19718309859154928
Split on feature emp_length_6 years. (133, 9)
--------------------------------------------------------------------
Subtree, depth = 13 (133 data points).
0.18045112781954886 0.18045112781954886
Split on feature home_ownership_RENT. (0, 133)
--------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 14 (133 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 13 (9 data points).
0.4444444444444444 0.4444444444444444
Split on feature home_ownership_REN

0.29254571026722925 0.29254571026722925
Split on feature term_ 60 months. (0, 2133)
--------------------------------------------------------------------
Subtree, depth = 9 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 9 (2133 data points).
0.29254571026722925 0.29254571026722925
Split on feature home_ownership_MORTGAGE. (1045, 1088)
--------------------------------------------------------------------
Subtree, depth = 10 (1045 data points).
0.261244019138756 0.261244019138756
Split on feature home_ownership_OTHER. (1044, 1)
--------------------------------------------------------------------
Subtree, depth = 11 (1044 data points).
0.2614942528735632 0.2614942528735632
Split on feature home_ownership_OWN. (879, 165)
--------------------------------------------------------------------
Subtree, depth = 12 (879 data points).
0.24687144482366324 0.246871444823663

0.3859030837004405 0.3859030837004405
Split on feature home_ownership_OWN. (1135, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (1135 data points).
0.3859030837004405 0.3859030837004405
Split on feature home_ownership_RENT. (1135, 0)
--------------------------------------------------------------------
Subtree, depth = 13 (1135 data points).
0.3859030837004405 0.3859030837004405
Split on feature emp_length_1 year. (1096, 39)
--------------------------------------------------------------------
Subtree, depth = 14 (1096 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 14 (39 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------

...

0.2619047619047619 0.2619047619047619
Split on feature home_ownership_OTHER. (40, 2)
--------------------------------------------------------------------
Subtree, depth = 14 (40 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 14 (2 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 13 (1 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 12 (1 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 11 (1 data points).
Stopping condition 1 reached. All data points have the same target value.
-----------------------------

0.3125 0.3125
Split on feature grade_B. (16, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (16 data points).
0.3125 0.3125
Split on feature grade_G. (16, 0)
--------------------------------------------------------------------
Subtree, depth = 13 (16 data points).
0.3125 0.3125
Split on feature term_ 60 months. (16, 0)
--------------------------------------------------------------------
Subtree, depth = 14 (16 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 12 (0 data points).
St

0.4956521739130435 0.4956521739130435
Split on feature grade_B. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 9 (230 data points).
0.4956521739130435 0.4956521739130435
Split on feature grade_G. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 10 (230 data points).
0.4956521739130435 0.4956521739130435
Split on feature term_ 60 months. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 11 (230 data points).
0.4956521739130435 0.4956521739130435
Split on feature home_ownership_OTHER. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (230 data points).
0.4956521739130435 0.4956521739130435
Split on feature home_ownership_OWN. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 13 (230 data points).
0.4956521739130435 0.4956521739130435
Split on feature home_owne

0.3738244514106583 0.3738244514106583
Split on feature grade_G. (1276, 0)
--------------------------------------------------------------------
Subtree, depth = 8 (1276 data points).
0.3738244514106583 0.3738244514106583
Split on feature term_ 60 months. (1276, 0)
--------------------------------------------------------------------
Subtree, depth = 9 (1276 data points).
0.3738244514106583 0.3738244514106583
Split on feature home_ownership_MORTGAGE. (855, 421)
--------------------------------------------------------------------
Subtree, depth = 10 (855 data points).
0.352046783625731 0.352046783625731
Split on feature home_ownership_OTHER. (849, 6)
--------------------------------------------------------------------
Subtree, depth = 11 (849 data points).
0.35217903415783275 0.35217903415783275
Split on feature home_ownership_OWN. (737, 112)
--------------------------------------------------------------------
Subtree, depth = 12 (737 data points).
0.35278154681139756 0.35278154681139756
S

0.3869379717755169 0.3869379717755169
Split on feature home_ownership_OTHER. (3037, 10)
--------------------------------------------------------------------
Subtree, depth = 11 (3037 data points).
0.3872242344418834 0.3872242344418834
Split on feature home_ownership_OWN. (2633, 404)
--------------------------------------------------------------------
Subtree, depth = 12 (2633 data points).
0.3904291682491455 0.3904291682491455
Split on feature home_ownership_RENT. (0, 2633)
--------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 13 (2633 data points).
0.3904291682491455 0.3904291682491455
Split on feature emp_length_1 year. (2392, 241)
--------------------------------------------------------------------
Subtree, depth = 14 (2392 data points).
Early stopping condition 1 reached. R

In [26]:
print("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data))
print("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data))
print("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data))

Training data, classification error (model 1): 0.40003761014399314
Training data, classification error (model 2): 0.3804266064904363
Training data, classification error (model 3): 0.3772566086395874


In [27]:
print("Training data, classification error (model 1):", evaluate_classification_error(model_1, validation_data))
print("Training data, classification error (model 2):", evaluate_classification_error(model_2, validation_data))
print("Training data, classification error (model 3):", evaluate_classification_error(model_3, validation_data))

Training data, classification error (model 1): 0.3981042654028436
Training data, classification error (model 2): 0.37774666092201636
Training data, classification error (model 3): 0.38140887548470487


In [28]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [29]:
print("Complexity of model_1 :",count_leaves(model_1))
print("Complexity of model_2 :",count_leaves(model_2))
print("Complexity of model_3 :",count_leaves(model_3))

Complexity of model_1 : 4
Complexity of model_2 : 39
Complexity of model_3 : 341


In [30]:
#ignoring this early stopping condition
model_4=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)

#just right
model_5=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=0)


#too positive
model_6=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=5)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
0.4963464431549538 0.4216365785514722
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
0.34923560663558495 0.34674184104955
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
0.34630563472922604 0.34630563472922604
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
0.33415902898191724 0.33415902898191724
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
0.30319510537049627 0.30319510537049627
Split on feature grade_D. (3826, 2058)
-------------------------------------------------------------

0.4056483835815474 0.3992008717762441
Split on feature grade_F. (21666, 358)
--------------------------------------------------------------------
Subtree, depth = 4 (21666 data points).
0.4008123326871596 0.39831994830610173
Split on feature grade_C. (14444, 7222)
--------------------------------------------------------------------
Subtree, depth = 5 (14444 data points).
0.349349210744946 0.34623373026862364
Split on feature grade_G. (14347, 97)
--------------------------------------------------------------------
Subtree, depth = 6 (14347 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (97 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (7222 data points).
0.4962614234284132 0.4745222929936306
Split on feature home_ownership_MORTGAGE. (4303, 2919)
----------------

0.4169325675388215 0.4169325675388215
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
0.4963464431549538 0.4216365785514722
Early stopping condition 3 reached. Minimum error reduction.


In [31]:
print("Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data))
print("Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data))
print("Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data))

Validation data, classification error (model 4): 0.37774666092201636
Validation data, classification error (model 5): 0.37774666092201636
Validation data, classification error (model 6): 0.503446790176648


In [32]:
print("Complexity of model_4 :",count_leaves(model_4))
print("Complexity of model_5 :",count_leaves(model_5))
print("Complexity of model_6 :",count_leaves(model_6))

Complexity of model_4 : 39
Complexity of model_5 : 12
Complexity of model_6 : 1


In [33]:
#too small
model_7=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)

#just right
model_8=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 2000, min_error_reduction=-1)


#may be too large
model_9=decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,min_node_size = 50000, min_error_reduction=-1)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
0.4963464431549538 0.4216365785514722
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
0.34923560663558495 0.34674184104955
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
0.34630563472922604 0.34630563472922604
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
0.33415902898191724 0.33415902898191724
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
0.30319510537049627 0.30319510537049627
Split on feature grade_D. (3826, 2058)
-------------------------------------------------------------

0.4056483835815474 0.3992008717762441
Split on feature grade_F. (21666, 358)
--------------------------------------------------------------------
Subtree, depth = 4 (21666 data points).
0.4008123326871596 0.39831994830610173
Split on feature grade_C. (14444, 7222)
--------------------------------------------------------------------
Subtree, depth = 5 (14444 data points).
0.349349210744946 0.34623373026862364
Split on feature grade_G. (14347, 97)
--------------------------------------------------------------------
Subtree, depth = 6 (14347 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (97 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (7222 data points).
0.4962614234284132 0.4745222929936306
Split on feature home_ownership_MORTGAGE. (4303, 2919)
----------------

0.4454840898539338 0.4175922288489697
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
0.417725321888412 0.4039055793991416
Split on feature grade_E. (22024, 1276)
--------------------------------------------------------------------
Subtree, depth = 3 (22024 data points).
0.4056483835815474 0.3992008717762441
Split on feature grade_F. (21666, 358)
--------------------------------------------------------------------
Subtree, depth = 4 (21666 data points).
0.4008123326871596 0.39831994830610173
Split on feature grade_C. (14444, 7222)
--------------------------------------------------------------------
Subtree, depth = 5 (14444 data points).
0.349349210744946 0.34623373026862364
Split on feature grade_G. (14347, 97)
--------------------------------------------------------------------
Subtree, depth = 6 (14347 data points).
Early stopping condition 1 reached. Reached maximum depth.
---------

In [34]:
print("Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data))
print("Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data))
print("Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data))

Validation data, classification error (model 7): 0.37774666092201636
Validation data, classification error (model 8): 0.3774235243429556
Validation data, classification error (model 9): 0.503446790176648


In [35]:
print("Complexity of model_7 :",count_leaves(model_7))
print("Complexity of model_8 :",count_leaves(model_8))
print("Complexity of model_9 :",count_leaves(model_9))

Complexity of model_7 : 39
Complexity of model_8 : 20
Complexity of model_9 : 1
